# Reuters Graph version

### Sources
- [Philipp tutorial](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial7/GNN_overview.html)
- [pytorch-geometric](https://pytorch-geometric.readthedocs.io/en/latest/)
- [nltk docs](https://www.nltk.org/book/ch02.html)

In [1]:
import torch
import numpy as np

from data_prep.reuters_graph import R8Graph, R52Graph

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.cuda("cpu")

In [2]:
r8 = R8Graph(device)

Prepare Reuters dataset
Compute tf.idf
Compute PMI scores
Generate edges
Generate masks
Generate feature matrix
Features mtx is 0.584769124 GBs in size


In [3]:
print(np.unique(r8.data.y[r8.data.train_mask].cpu(), return_counts=True))
print(np.unique(r8.data.y[r8.data.val_mask].cpu(), return_counts=True))

(array([0, 1, 2, 3, 4, 5, 6, 7]), array([262,  42, 461,  15,  39,  38,  36, 107]))
(array([0, 1, 2, 4, 5, 7]), array([35,  5, 49,  2,  4,  5]))


In [4]:
print(sum(r8.data.val_mask * r8.data.train_mask * r8.data.test_mask))
print(sum(r8.data.train_mask))
print(sum(r8.data.val_mask))
print(sum(r8.data.test_mask))

tensor(0, device='cuda:0')
tensor(1000, device='cuda:0')
tensor(100, device='cuda:0')
tensor(100, device='cuda:0')


In [5]:
# GraphConv, GATConv
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GraphConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(len(r8.iton), 200)
        self.conv2 = GCNConv(200, 8)

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x

def eval(model, data, mask):
    _, pred = model(data).max(dim=1)
    #print(pred[mask])
    #print(data.y[mask])
    correct = pred[mask].eq(data.y[mask]).sum().item()
    acc = correct / mask.sum()
    print('Accuracy: {:.4f}'.format(acc))

In [6]:
model = Net()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)#, weight_decay=5e-4)
r8.data.to(device)

Data(edge_attr=[1209458], edge_index=[2, 1209458], test_mask=[12091], train_mask=[12091], val_mask=[12091], x=[12091, 12091], y=[12091])

In [7]:
from tqdm.notebook import tqdm

model.train()
for epoch in tqdm(range(40)):
    optimizer.zero_grad()
    out = model(r8.data)
    # We might want to use the "weight" parameter for the loss with unbalanced dataset
    # since with a low learning rate the model just assigns every doc to class "earn"
    # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
    loss = F.cross_entropy(out[r8.data.train_mask], r8.data.y[r8.data.train_mask])
    print('Loss:', loss.item())
    loss.backward()
    optimizer.step()
    print(np.unique(out[r8.data.train_mask].max(dim=1)[1].detach().cpu().numpy(), return_counts=True))
    #print(np.unique(r8.data.y[r8.data.train_mask].detach().cpu().numpy(), return_counts=True))
    eval(model, r8.data, r8.data.val_mask)

Loss: 2.0793890953063965
(array([0, 1, 2, 3, 4, 5, 6, 7]), array([522, 126,  35,  60,   9,   3, 220,  25]))
Accuracy: 0.4900
Loss: 1.600253701210022
(array([2]), array([1000]))
Accuracy: 0.4900
Loss: 1.4529918432235718
(array([2]), array([1000]))
Accuracy: 0.4900
Loss: 1.38367760181427
(array([2]), array([1000]))
Accuracy: 0.5200
Loss: 1.2150911092758179
(array([0, 2]), array([ 34, 966]))
Accuracy: 0.8200
Loss: 1.09518563747406
(array([0, 2]), array([467, 533]))
Accuracy: 0.8200
Loss: 0.9767016172409058
(array([0, 2]), array([473, 527]))
Accuracy: 0.8300
Loss: 0.8734109401702881
(array([0, 2]), array([443, 557]))
Accuracy: 0.8100
Loss: 0.7946096062660217
(array([0, 2]), array([454, 546]))
Accuracy: 0.8300
Loss: 0.7308882474899292
(array([0, 2, 7]), array([469, 530,   1]))
Accuracy: 0.8200
Loss: 0.6623002290725708
(array([0, 2, 7]), array([428, 522,  50]))
Accuracy: 0.8500
Loss: 0.5993506908416748
(array([0, 2, 7]), array([290, 536, 174]))
Accuracy: 0.8600
Loss: 0.5443897843360901
(arra

In [8]:
eval(model, r8.data, r8.data.test_mask)

Accuracy: 0.9200
